In [1]:
import pandas as pd
import numpy as np

In [2]:
practice_data = pd.read_csv("data/GPsize.csv")
practice_address_data = pd.read_csv("data/T201912ADDR BNFT.csv", names = ["date", "Code", "Name", "Name2", "Add", "Loc", "Place", "Postcode"])
pollution_data = pd.read_csv("data/mappm252019g.csv")
prescription_data = pd.read_csv("data/T201912PDPI BNFT.csv")

FileNotFoundError: [Errno 2] File b'data/GPsize.csv' does not exist: b'data/GPsize.csv'

In [ ]:
prescription_data.rename(columns = {"PRACTICE":"Code"}, inplace = True)
prescription_data = prescription_data.merge(practice_data.drop(columns=["Practice"]))

In [ ]:
import pgeocode
from convertbng.util import convert_bng
import tqdm

practice_address_data = practice_address_data.drop(columns=["date", "Name", "Name2", "Add", "Loc", "Place"])

nomi = pgeocode.Nominatim('gb')
practice_address_data["x"], practice_address_data["y"] = 0, 0


for i in tqdm.tqdm(range(len(practice_address_data))):
    postcode = practice_address_data["Postcode"][i]
    if pd.isnull(postcode): continue
    loc = nomi.query_postal_code(practice_address_data["Postcode"][i])
    x,y = convert_bng(loc["longitude"], loc["latitude"])
    if pd.isnull(x) or pd.isnull(y): continue
    practice_address_data["x"][i] = int(round(x[0], -3)) + 500
    practice_address_data["y"][i] = int(round(y[0], -3)) + 500
    
prescription_data = prescription_data.merge(practice_address_data)

In [ ]:
pd.isnull(practice_address_data["Postcode"][4807])

In [ ]:
prescription_data["Total List Size"] = prescription_data["Total List Size"].str.replace(',', '').astype(float)
prescription_data

In [ ]:
prescription_data.sum("QUANTITY")

In [ ]:
prescription_data["RATE"] = prescription_data["QUANTITY"] / prescription_data["Total List Size"]

In [ ]:
prescription_data = prescription_data.merge(pollution_data)
prescription_data = prescription_data.replace('MISSING',np.NaN)
prescription_data["pm252019g"] = pd.to_numeric(prescription_data["pm252019g"], downcast="float")

In [ ]:
prescription_data["BNF CHAPTER"] = prescription_data["BNF CODE"].astype(str).str[0:2]

In [ ]:
prescription_data

In [ ]:
from matplotlib import pyplot
asthma = prescription_data.loc[prescription_data["BNF CHAPTER"] == "03"]
asthma = asthma.loc[prescription_data["RATE"] > 1]
pyplot.scatter(asthma['pm252019g'].values, asthma['RATE'].values)

In [ ]:
import scipy.stats
asthma = asthma.dropna()
scipy.stats.pearsonr(asthma['pm252019g'].values, asthma['RATE'].values)

In [ ]:
asthma['pm252019g'].values


In [ ]:
practice_data

In [ ]:
prescription_data["BNF CODE"][6266000]